In [5]:
import cv2
import numpy as np
import random

def add_bouncing_watermark(input_video_path, output_video_path, text="Your Watermark", font_scale=1, alpha=0.5):
    # Open the video
    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    
    # Define the codec and create a VideoWriter object to save the video
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))
    
    # Prepare text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_thickness = 2
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    
    # Random initial position
    text_x = random.randint(0, frame_width - text_size[0])
    text_y = random.randint(0, frame_height - text_size[1])
    
    # Random direction
    direction_x = random.choice([-1, 1])
    direction_y = random.choice([-1, 1])
    step = 5  # Speed of the movement
    
    # Process each frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Create a transparent overlay for the watermark
        overlay = frame.copy()
        
        # Add text moving across the frame
        cv2.putText(overlay, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
        
        # Update the position of the text for the next frame
        text_x += step * direction_x
        text_y += step * direction_y

        # Bounce off the edges
        if text_x + text_size[0] > frame_width or text_x < 0:
            direction_x *= -1
        if text_y + text_size[1] > frame_height or text_y < 0:
            direction_y *= -1
        
        # Write the frame with the watermark to the new video
        out.write(frame)
    
    # Release everything when done
    cap.release()
    out.release()

# Example usage
input_video_path = 'test.mp4'
output_video_path = 'watermarked.mp4'
add_bouncing_watermark(input_video_path, output_video_path, text="Hello World", font_scale=1, alpha=0.5)
